In [1]:
import nest_asyncio
nest_asyncio.apply()
import cmdstanpy as sp
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd
import matplotlib.pyplot as plt
plt.ioff()
from cycler import cycler
import matplotlib.patches as mpatches
import seaborn as sns
import arviz as az
import scipy.stats as stats
import ROOT as r
import time
plt.rcParams["xtick.minor.visible"] =  True
plt.rcParams["ytick.minor.visible"] =  True
plt.rcParams["mathtext.fontset"]="cm"
plt.rcParams['errorbar.capsize'] = 3
plt.rcParams['mathtext.rm'] = 'serif'
plt.rcParams['font.family']='serif'
plt.rcParams['font.size']=22
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.linestyle']=':'
plt.rcParams['grid.color']='black'
plt.rcParams['axes.prop_cycle'] = cycler(color=['black', 'red', 'blue', 'green'])

import arviz as az
import warnings
warnings.filterwarnings('ignore')
%jsroot on

/home/jakob/.local/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Welcome to JupyROOT 6.26/04


In [2]:
def fit(nsamples):
    cols=[f'ebin{i:02d}costbin{j:02d}' for i in range(3) for j in range(6)]
    diagnostics_df=pd.DataFrame(columns=cols,index=['sigma_median','mcse','rhat'])
    sigma_df=pd.DataFrame(columns=cols)
    sigma_2pi0_df=pd.DataFrame(columns=cols)
    #read bkg data
    df_bkg=pd.read_csv('./2pi0_bkg_costp.txt',sep=',',index_col=0)
    df_fr=pd.read_csv('./new_2pi0_bkg_fr.txt',sep=',',index_col=0)
    for i in range(3):#no. of energy bins
        for j in range(6):#no. of angle bins
            #read data
            df=pd.read_csv(f"ebin{i:02d}/ebin{i:02d}costbin{j:02d}.txt",sep="\t")
            df.columns=['pol','phi','weight']
            #these are prompt peak events
            prmpt=df.loc[df['weight']==1]
            prmpt=prmpt.reset_index(drop=True)
            #sideband
            side=df.loc[df['weight']!=1]
            side=side.reset_index(drop=True)
            nprmpt=len(prmpt)
            total_nside=(len(side))
            #weights used in the data, including 1--> exterminate 1
            weights=pd.unique(df['weight'].values)
            weights=np.array(weights[weights!=1])
            #sideband events are identified by the corresponding weights
            nside=np.array([len(df.loc[df['weight']==weights[i]])for i in range(len(weights))])
            #fraction of signal in prompt peak, note that the weights are already negative (TO BE CHECKED W/ FARAHS DATA!)
            f=(nprmpt+np.sum(nside*weights))/nprmpt
            #print(f)
            #read fraction of bkg in data
            fr=df_fr[f'ebin{i:02d}costbin{j:02d}'].values[0]
            #read bkg contribution, note: take costheta from last bin to first, since its binned in costheta p
            #also factor of -1 needed since I_c = -Sigma
            sigma_2pi0=-1*df_bkg[f'ebin{i:02d}costbin{-j+5:02d}']['sigma']
            sigma_2pi0_err=df_bkg[f'ebin{i:02d}costbin{-j+5:02d}']['sigma_err']
            
            stan_data={
                'N':nprmpt, #no. of prompt peak events and corresponding pol and phi values
                'phi_prmpt':list(prmpt['phi'].values),
                'pol_prmpt':list(prmpt['pol'].values),
                'M':total_nside,#no. of sideband events and..
                'phi_side':list(side['phi'].values),
                'pol_side':list(side['pol'].values),
                'f':f, #fraction of signal in prmpt peak
                'f_s': 1-fr, #fraction of etap events
                'f_b': fr, #fraction of 2pi0 events
                'sigma_2pi0_meas': sigma_2pi0,#measured value of sigma_2pi0 
                'dsigma_2pi0_meas': sigma_2pi0_err #w/ stat error
            }
            #now the stan model and mcmc
            model=sp.CmdStanModel(stan_file='toyMC_stan.stan')
            model.compile()
            fitobj=model.sample(data=stan_data,iter_sampling=nsamples,inits=0)
            summary=fitobj.summary()
            samples=fitobj.draws_pd()
            #get mcmc diagnostics
            median=summary['50%']['sigma']
            mcse=(az.mcse(np.transpose(fitobj.draws(concat_chains=False)[:,:,7]),method='median'))
            rhat=(summary['R_hat']['sigma'])
            tmp_list=[median,mcse,rhat]
            currbin=f"ebin{i:02d}costbin{j:02d}"
            diagnostics_df[currbin]=tmp_list
            sigma_df[currbin]=samples['sigma']
            sigma_2pi0_df[currbin]=samples['sigma_2pi0']
            print(currbin)
    return diagnostics_df, sigma_df, sigma_2pi0_df
        
def fit_bin(nsamples,i,j): #fit only one bin
    #read data
    df=pd.read_csv(f"ebin{i:02d}/ebin{i:02d}costbin{j:02d}.txt",sep="\t")    
    df.columns=['pol','phi','weight']
    #these are prompt peak events
    prmpt=df.loc[df['weight']==1]
    prmpt=prmpt.reset_index(drop=True)
    #sideband
    side=df.loc[df['weight']!=1]
    side=side.reset_index(drop=True)
    nprmpt=len(prmpt)
    total_nside=(len(side))
    #weights used in the data, including 1--> exterminate 1
    weights=pd.unique(df['weight'].values)
    weights=np.array(weights[weights!=1])
    #sideband events are identified by the corresponding weights
    nside=np.array([len(df.loc[df['weight']==weights[i]])for i in range(len(weights))])
    #fraction of signal in prompt peak, note that the weights are already negative (TO BE CHECKED W/ FARAHS DATA!)
    f=(nprmpt+np.sum(nside*weights))/nprmpt
    print(f)
    stan_data={
        'N':nprmpt, #no. of prompt peak events and corresponding pol and phi values
        'phi_prmpt':list(prmpt['phi'].values),
        'pol_prmpt':list(prmpt['pol'].values),
        'M':total_nside,#no. of sideband events and..
        'phi_side':list(side['phi'].values),
        'pol_side':list(side['pol'].values),
        'f':f #fraction of signal in prmpt peak
    }
    print(nprmpt, total_nside)
    #now the stan model and mcmc
    model=sp.CmdStanModel(stan_file='old_stan.stan')
    model.compile()
    fitobj=model.sample(data=stan_data,iter_sampling=nsamples,inits=0)
    summary=fitobj.summary()
    samples=fitobj.draws_pd()
    return samples,summary

In [3]:
dfs=fit(nsamples=5000)
diagnostics=dfs[0]
sigma=dfs[1]
sigma_2pi0=dfs[2]
diagnostics.to_csv('alt_diagnostics.csv')
sigma.to_csv('alt_sigma.csv')
sigma_2pi0.to_csv('sigma_2pi0.csv')

INFO:cmdstanpy:compiling stan file /home/jakob/Documents/Studium/master_thesis/bayes/etap_event_based_fit/toyMC_stan.stan to exe file /home/jakob/Documents/Studium/master_thesis/bayes/etap_event_based_fit/toyMC_stan
INFO:cmdstanpy:compiled model executable: /home/jakob/Documents/Studium/master_thesis/bayes/etap_event_based_fit/toyMC_stan
INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:CmdStan start procesing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

...

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

INFO:cmdstanpy:CmdStan done processing.



ebin02costbin05


In [5]:
dfs=fit_bin(nsamples=1000,i=2,j=5)
samples=dfs[0]
#diagnostics.to_csv('diagnostics.csv')
samples.to_csv('samples_ebin02costbin05.csv')

INFO:cmdstanpy:compiling stan file /home/jakob/Documents/Studium/master_thesis/bayes/etap_event_based_fit/old_stan.stan to exe file /home/jakob/Documents/Studium/master_thesis/bayes/etap_event_based_fit/old_stan


0.9362218045112782
266 261


INFO:cmdstanpy:compiled model executable: /home/jakob/Documents/Studium/master_thesis/bayes/etap_event_based_fit/old_stan
INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:CmdStan start procesing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

INFO:cmdstanpy:CmdStan done processing.


In [6]:
summary=dfs[1]
summary.to_csv('summary_ebin02costbin05.csv')